In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import matplotlib.pyplot as plt


In [30]:
# Load CSV file including 100k events
print("Loading dataset...")
df = pd.read_csv('/home/edaerdogan/Desktop/dimuonai/dimuon.csv')


Loading dataset...


In [31]:
#filtering and labeling 
signal_df = df[df['Q1'] * df['Q2'] < 0].copy()
background_df = df[df['Q1'] * df['Q2'] >= 0].copy() 

signal_df['target'] = 0
background_df['target'] = 1

filtered_df = pd.concat([signal_df, background_df], ignore_index=True)



In [32]:
signal_count = len(df[df['Q1'] * df['Q2'] < 0])
background_count = len(df[df['Q1'] * df['Q2'] >= 0])

total_signal = signal_count
total_background = background_count
total_events = total_signal + total_background

# Calculate percentages
signal_percentage = total_signal / total_events * 100
background_percentage = total_background / total_events * 100

print(f'Total Signal Events: {total_signal}')
print(f'Total Background Events: {total_background}')
print(f'Signal Percentage: {signal_percentage}%')
print(f'Background Percentage: {background_percentage}%')

Total Signal Events: 62214
Total Background Events: 37786
Signal Percentage: 62.214000000000006%
Background Percentage: 37.785999999999994%


In [33]:
features = filtered_df[['pt1', 'pt2', 'eta1', 'eta2', 'phi1', 'phi2', 'Q1', 'Q2']]
target = filtered_df['target']

X = features.to_numpy(dtype='float')
Y = target.to_numpy(dtype='float')

In [34]:
x = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(Y, dtype=torch.float32).reshape(-1, 1)
print(x)
print(y)

tensor([[ 6.2360,  2.3905, -0.5848,  ..., -2.2764, -1.0000,  1.0000],
        [ 8.9484,  6.7821, -1.3530,  ..., -0.3814, -1.0000,  1.0000],
        [ 4.0910,  4.8186,  1.2463,  ...,  2.2493, -1.0000,  1.0000],
        ...,
        [11.4290,  1.1869,  1.8963,  ..., -1.6911, -1.0000, -1.0000],
        [ 9.4401,  2.8993,  1.3652,  ..., -0.5243,  1.0000,  1.0000],
        [ 3.3954, 12.6496,  2.3445,  ..., -1.8854,  1.0000,  1.0000]])
tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])


In [35]:
# define the model
# define the model
model = nn.Sequential(
    nn.Linear(8, 100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 20),
    nn.ReLU(),
    nn.Linear(20, 1),
    nn.Sigmoid()
)
print(model)
# train the model
loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001) #lr değiştirrilebilir mi 
 
n_epochs = 20 #e:15 b:32 accuracy: 0.377, e:15 b:64 a:0.377, e:15 b:128 a:0.377, e:30, b:128 a:0.97; e:20 b:128 a:0.99(best result)
batch_size = 128
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = x[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
 
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(x)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")


Sequential(
  (0): Linear(in_features=8, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=20, bias=True)
  (5): ReLU()
  (6): Linear(in_features=20, out_features=1, bias=True)
  (7): Sigmoid()
)


Finished epoch 0, latest loss 4.1806888475548476e-05
Finished epoch 1, latest loss 0.0005317169707268476
Finished epoch 2, latest loss 8.621768211014569e-05
Finished epoch 3, latest loss 0.003441666718572378
Finished epoch 4, latest loss 3.7849276850465685e-06
Finished epoch 5, latest loss 0.0004483017837628722
Finished epoch 6, latest loss 0.002661291277036071
Finished epoch 7, latest loss 5.1632290706038475e-05
Finished epoch 8, latest loss 6.906952330609784e-05
Finished epoch 9, latest loss 0.00010955195466522127
Finished epoch 10, latest loss 0.00041529370355419815
Finished epoch 11, latest loss 0.00023112961207516491
Finished epoch 12, latest loss 4.291659024602268e-06
Finished epoch 13, latest loss 9.445937030250207e-05
Finished epoch 14, latest loss 1.937151523634384e-07
Finished epoch 15, latest loss 3.166497890561004e-07
Finished epoch 16, latest loss 3.241004264964431e-07
Finished epoch 17, latest loss 0.0
Finished epoch 18, latest loss 0.0
Finished epoch 19, latest loss 2.04